<a href="https://colab.research.google.com/github/marcelo-vitti/coffee_and_currency_rates_case/blob/main/Pismo_Analytics_Engineer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get Currency Rates to SQLite Table

In [1]:
import pandas as pd
import requests
import json
import datetime

from datetime import date, timedelta

In [40]:
API_KEY = "58b5b643ff774a55babfbc8c21d66914"
START_DATE = date(2021, 1, 1)
END_DATE = date(2023, 3, 29)
CURRENCY_SOURCE = "USD"
DESIRED_CURRENCIES = ["BRL", "CLP", "EUR"]

In [41]:
def enable_log_messages():
        
  import logging

  logger = logging.getLogger()
  logger.setLevel(logging.INFO)

  return logging

In [42]:
def daterange(start_date, end_date):

    for n in range(int((end_date - start_date).days)):
      
        yield start_date + timedelta(n)

In [43]:
data = []

for single_date in daterange(START_DATE, END_DATE):

  api_call_date = single_date.strftime("%Y-%m-%d")

  logging = enable_log_messages()
  logging.info(f"[INFO] Getting currency rate for date {api_call_date}.")

  url = f"https://openexchangerates.org/api/historical/{api_call_date}.json?app_id={API_KEY}&show_alternative=false&prettyprint=false"

  payload = {}
  headers= {
    "apikey": API_KEY
  }

  response = requests.request("GET", url, headers=headers, data = payload)

  response_json = response.json()

  # On this for loop I am creating the table in the format that the challenge suggests "Esses dados devem ser 
  # estruturados (Moeda, Câmbio e Data)", but, maybe a better approach to make it easier to query would be 
  # to have just one line per date with all currencies instead of one line per date per currency.
  for key, value in response_json["rates"].items():

    if key in DESIRED_CURRENCIES:

      return_dict = {
        "activity_date": api_call_date,
        "currency": key,
        "rate": value,
      } 

      data.append(return_dict)


INFO:root:[INFO] Getting currency rate for date 2023-03-28.


In [44]:
df_currency_rates = pd.DataFrame(data)
df_currency_rates

,activity_date,currency,rate
0,2023-03-28,BRL,5.165900
1,2023-03-28,CLP,796.825397
2,2023-03-28,EUR,0.922607


In [31]:
df_currency_rates["activity_date"] = df_currency_rates["activity_date"].astype('datetime64[ns]')

In [36]:
df_currency_rates["is_month_end"] = df_currency_rates.activity_date.apply(lambda x: True if x.is_month_end else False)
df_currency_rates["is_year_end"] = df_currency_rates.activity_date.apply(lambda x: True if x.is_year_end else False)

In [7]:
import sqlite3

conn = sqlite3.connect("coffee_and_currencies.db")

In [8]:
df_currency_rates.to_sql("currency_rates_data", conn, if_exists="replace")

2451

In [9]:
conn.execute(
    """
    create table IF NOT EXISTS currency_rates as 
    select * from currency_rates_data
    """)

In [10]:
conn.commit()


In [11]:
conn.close()

# Get Coffee to SQLite Table

In [12]:
coffee = pd.read_csv("https://raw.githubusercontent.com/marcelo-vitti/coffee_and_currency_rates_case/main/external_sources/coffee.csv")

In [13]:
conn = sqlite3.connect("coffee_and_currencies.db")

coffee.to_sql("coffee_data", conn, if_exists="replace")

conn.execute(
    """
    create table IF NOT EXISTS coffee as 
    select * from coffee_data
    """)

conn.commit()


In [14]:
conn.close()

# Get SQL Queries results

In [15]:
import base64
import requests

coffee_per_year_and_rates_link = "https://raw.githubusercontent.com/marcelo-vitti/coffee_and_currency_rates_case/main/sql/highest_coffee_volume_and_currency_closing.sql"
req = requests.get(coffee_per_year_and_rates_link)
coffee_per_year_and_rates_sql = req.text

highest_coffee_volume_and_currency_closing_link = "https://raw.githubusercontent.com/marcelo-vitti/coffee_and_currency_rates_case/main/sql/highest_coffee_volume_and_currency_closing.sql"
req = requests.get(highest_coffee_volume_and_currency_closing_link)
highest_coffee_volume_and_currency_closing_sql = req.text

negotiated_coffee_average_link = "https://raw.githubusercontent.com/marcelo-vitti/coffee_and_currency_rates_case/main/sql/highest_coffee_volume_and_currency_closing.sql"
req = requests.get(negotiated_coffee_average_link)
negotiated_coffee_average_sql = req.text


In [16]:
conn = sqlite3.connect("coffee_and_currencies.db")

**coffee_per_year_and_rates:** Maior volume negociado de café no dia e as cotações de fechamento

In [17]:
df_coffee_per_year_and_rates = pd.read_sql_query(coffee_per_year_and_rates_sql, conn)
df_coffee_per_year_and_rates

,activity_date,higher_coffee_volume,brl_usd_rate,clp_usd_rate,eur_usd_rate
0,2021-01-04,129.40,None,None,None
1,2021-01-05,126.55,None,None,None
2,2021-01-06,125.55,None,None,None
3,2021-01-07,122.75,None,None,None
4,2021-01-08,124.05,None,None,None
...,...,...,...,...,...
414,2022-08-25,246.90,None,None,None
415,2022-08-26,245.25,None,None,None
416,2022-08-29,240.55,None,None,None
417,2022-08-30,241.75,None,None,None


**highest_coffee_volume_and_currency_closing:** Total de café negociado por ano e as cotações

In [18]:
df_highest_coffee_volume_and_currency_closing = pd.read_sql_query(highest_coffee_volume_and_currency_closing_sql, conn)
df_highest_coffee_volume_and_currency_closing

,activity_date,higher_coffee_volume,brl_usd_rate,clp_usd_rate,eur_usd_rate
0,2021-01-04,129.40,None,None,None
1,2021-01-05,126.55,None,None,None
2,2021-01-06,125.55,None,None,None
3,2021-01-07,122.75,None,None,None
4,2021-01-08,124.05,None,None,None
...,...,...,...,...,...
414,2022-08-25,246.90,None,None,None
415,2022-08-26,245.25,None,None,None
416,2022-08-29,240.55,None,None,None
417,2022-08-30,241.75,None,None,None


**negotiated_coffee_average:** Média de volume negociado mensal e anual 

In [19]:
df_negotiated_coffee_average = pd.read_sql_query(negotiated_coffee_average_sql, conn)
df_negotiated_coffee_average

,activity_date,higher_coffee_volume,brl_usd_rate,clp_usd_rate,eur_usd_rate
0,2021-01-04,129.40,None,None,None
1,2021-01-05,126.55,None,None,None
2,2021-01-06,125.55,None,None,None
3,2021-01-07,122.75,None,None,None
4,2021-01-08,124.05,None,None,None
...,...,...,...,...,...
414,2022-08-25,246.90,None,None,None
415,2022-08-26,245.25,None,None,None
416,2022-08-29,240.55,None,None,None
417,2022-08-30,241.75,None,None,None


In [20]:
conn.close()

# Save SQL queries results in Google Drive

In [21]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
base_path = "/content/drive/MyDrive/PismoCase/csv"

In [46]:
df_coffee_per_year_and_rates.to_csv(f"{base_path}/coffee_per_year_and_rates.csv", index=False)
df_highest_coffee_volume_and_currency_closing.to_csv(f"{base_path}/highest_coffee_volume_and_currency_closing.csv", index=False)
df_negotiated_coffee_average.to_csv(f"{base_path}/negotiated_coffee_average.csv", index=False)
df_currency_rates.to_csv(f"{base_path}/currency_rates.csv", index=False)